<a href="https://colab.research.google.com/github/UrviSinghal28/Machine-Learning-with-Python/blob/main/Tensorflow/RNN/Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Play Generator  
Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.

This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
#for loading data from device
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [9]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [12]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


**Creating Loss Function**

In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 2.6689619e-03  7.1242275e-03  9.2442636e-04 ... -2.1696275e-03
   -6.9645810e-04 -7.3643215e-04]
  [ 5.4520782e-04  5.1136953e-03 -8.8859597e-05 ... -1.9401346e-03
    4.2081494e-03  3.3043278e-04]
  [ 6.3985358e-03  4.5895032e-03 -3.5562159e-03 ...  3.1026383e-03
    2.9060799e-03 -1.1590844e-03]
  ...
  [-9.1450177e-03 -3.4011686e-03  2.1874057e-03 ... -5.0105499e-03
    1.9240610e-03 -1.4445326e-02]
  [-1.4952555e-03 -3.8595963e-03 -9.4987813e-04 ... -2.3878780e-03
    1.5133580e-03 -1.3805853e-02]
  [-1.2338888e-03 -4.1050073e-03 -3.4513997e-03 ... -5.3489977e-03
    2.1666752e-03 -1.1913513e-02]]

 [[ 4.5000026e-03 -5.2941940e-03  3.1399883e-03 ...  5.2617845e-04
    1.5793546e-03  3.0207331e-03]
  [ 1.4424168e-04 -8.4293587e-03  5.0746294e-04 ...  2.1664940e-03
    2.7141604e-03  1.0496870e-02]
  [ 5.7156489e-04 -5.1313969e-03 -1.2207552e-04 ...  6.7694923e-03
   -2.3117489e-03  9.4853062e-03]
  ...
  [-5.3950637e-03  3.0619935e-03 -3.8545099e-04 ... -4.1818828e

In [17]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 2.6689619e-03  7.1242275e-03  9.2442636e-04 ... -2.1696275e-03
  -6.9645810e-04 -7.3643215e-04]
 [ 5.4520782e-04  5.1136953e-03 -8.8859597e-05 ... -1.9401346e-03
   4.2081494e-03  3.3043278e-04]
 [ 6.3985358e-03  4.5895032e-03 -3.5562159e-03 ...  3.1026383e-03
   2.9060799e-03 -1.1590844e-03]
 ...
 [-9.1450177e-03 -3.4011686e-03  2.1874057e-03 ... -5.0105499e-03
   1.9240610e-03 -1.4445326e-02]
 [-1.4952555e-03 -3.8595963e-03 -9.4987813e-04 ... -2.3878780e-03
   1.5133580e-03 -1.3805853e-02]
 [-1.2338888e-03 -4.1050073e-03 -3.4513997e-03 ... -5.3489977e-03
   2.1666752e-03 -1.1913513e-02]], shape=(100, 65), dtype=float32)


In [18]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 0.00266896  0.00712423  0.00092443 -0.00180136 -0.00112374 -0.0093459
 -0.00467908 -0.00686646 -0.00191638  0.00337418  0.00201906 -0.00727935
  0.00363835 -0.00627986 -0.00431121 -0.0022136  -0.0033052  -0.00414689
 -0.0035548  -0.00329995 -0.01196228  0.00184924  0.00321342  0.00263478
 -0.0013077  -0.001501    0.00013483  0.00663195 -0.00427773 -0.0082995
  0.00434565  0.00192302  0.00327656  0.00010063 -0.0007845  -0.0055686
  0.00410312 -0.00101522 -0.00631332 -0.00013281 -0.00142675 -0.00151265
  0.00043202 -0.00098441  0.00062713 -0.00112656  0.00539808  0.00060397
  0.0031465   0.00376959 -0.00309899  0.00020527 -0.00177377  0.00750922
 -0.00050925  0.00402068  0.00043059  0.00266882 -0.00676315  0.00366074
 -0.00193647 -0.00480805 -0.00216963 -0.00069646 -0.00073643], shape=(65,), dtype=float32)


In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"MI'bE?RWgHjV?Bt33KF!:YD.aDRPAh:suGYKeKb eDLinW,Krij,a,xWe.rGVYN!IIoRMsnrdb'KuFmwpgxpiZ&uTMPSjbGcaXEv"

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

**Compiling the Model**

In [21]:
model.compile(optimizer='adam', loss=loss)

**Creating Checkpoint**

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
#In this case there is no fear of over-fitting so more the number of epochs better the model will be
history = model.fit(data, epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5
172/172 [==============================] - 31s 173ms/step - loss: 1.5501
Epoch 2/5
172/172 [==============================] - 31s 173ms/step - loss: 1.4827
Epoch 3/5
172/172 [==============================] - 31s 173ms/step - loss: 1.4330
Epoch 4/5
172/172 [==============================] - 31s 172ms/step - loss: 1.3961
Epoch 5/5
172/172 [==============================] - 31s 172ms/step - loss: 1.3654


In [25]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [26]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

**Generating Text**

In [29]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeow my thrick in'er
comm them thank your loss betwn not, Bonalunation
With would have revengent, yet. The world a man ten hope and this ansile
Pity men: never well neat, and see them, good and Marcius brought
I gove me so not out eyes, I am:
All foller unralmad wills,
As thorucently touch my monies munder perpetes
Cault from the supt the Volsence,
Which doth he becless a sacraper,
Wherefore I must feak Bulint o' the enough.

LUCENTIO:
My lord!
You skeep, and now she wouldest us you?

AUTOLYCUS:
I pray the dourness, and dear bringshed:
And thites we after men of England's ve
bears me here of a kind, resign; whose field
Her painces soldier, come to me, noble as go
Of the emindness, darns wounded the wanch of Beauth.

HASTINGS:
Ah, pourted, nor my soke
Writch thou says my consence.

LADY GREY:

